In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 12 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

09 January 2023 ; 17:56:20


In [3]:
scrape_year = 2023

# Data preparation

In [4]:
# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv').groupby('roy').get_group(0) # Load up-to-date data, omit rookies
# Load big data file (to compute careers and comparison)
final_data = pd.read_csv(path + 'final_data.csv')
final_data = final_data[final_data['Year'] < scrape_year]
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
1,Bobby Portis,PF,27,MIL,39,9,26.5,5.9,12.0,0.496,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,1
2,Brook Lopez,C,34,MIL,38,38,30.3,5.5,10.8,0.507,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,0
3,George Hill,PG,36,MIL,31,0,19.8,1.7,3.8,0.437,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,1
4,Giannis Antetokounmpo,PF,28,MIL,33,33,33.6,11.3,21.3,0.530,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,0
5,Grayson Allen,SG,27,MIL,36,34,27.4,3.3,7.7,0.430,...,1.21,112.47,111.2,1.27,1.11,112.45,111.24,1.21,0,0


In [5]:
# For each player, determine his career lifetime, his career high in points, assists, rebounds and minutes played
# as well as his previous playing year
def years_in_league(df):
    player = list(df['Player'])[0] # get player name
    temp = final_data.groupby('Player').get_group(player).sort_values('Year') # get player data sorted by year
    #if len(list(temp['Year'])) < 2 : # if it's his first year set dummy previous year (easy to cut)
      #  df['prev_year'] = [0]
    #else : # else save previous playing year
    df['prev_year'] = [list(temp['Year'])[-1]]

    df['lifetime'] = [len(temp) + 1] # compute career lifetime
    
    df_prev = temp[temp['Year'] < scrape_year] # get previous data
    # Compute career high
    df['PTS_high'] = [df_prev['PTS'].max()]
    df['AST_high'] = [df_prev['AST'].max()]
    df['TRB_high'] = [df_prev['TRB'].max()]
    df['MP_high'] = [df_prev['MP'].max()]

    return df

data = data.groupby('Player').apply(years_in_league)
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,DRtg/A,NRtg/A,roy,smoy,prev_year,lifetime,PTS_high,AST_high,TRB_high,MP_high
1,Bobby Portis,PF,27,MIL,39,9,26.5,5.9,12.0,0.496,...,111.24,1.21,0,1,2022,8,14.6,1.7,9.1,28.2
2,Brook Lopez,C,34,MIL,38,38,30.3,5.5,10.8,0.507,...,111.24,1.21,0,0,2022,15,20.7,2.3,8.6,36.9
3,George Hill,PG,36,MIL,31,0,19.8,1.7,3.8,0.437,...,111.24,1.21,0,1,2022,15,16.9,5.1,4.2,34.5
4,Giannis Antetokounmpo,PF,28,MIL,33,33,33.6,11.3,21.3,0.530,...,111.24,1.21,0,0,2022,10,29.9,5.9,13.6,36.7
5,Grayson Allen,SG,27,MIL,36,34,27.4,3.3,7.7,0.430,...,111.24,1.21,0,0,2022,5,11.1,2.2,3.4,27.3


In [6]:
# Only keep players having played in 2022 and merge with last year data : may be a case where MIP might be relative to a further year !!!
data = data.groupby('prev_year').get_group(scrape_year-1) 
data = data.merge(final_data.rename(columns = {'Year' : 'prev_year'}), on = ['Player', 'prev_year'], suffixes = ['', '_prev'], how = 'inner')

### FEATURES ####
# Calculate the difference in stats for all numerical features
features =  ['G','GS','MP','FG','FGA','FG%','3P','3PA','3P%','2P','2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST',
             'STL','BLK','TOV','PF','PTS','PER','TS%','3PAr','FTr','ORB%','DRB%','TRB%','AST%','STL%','BLK%','TOV%','USG%',
             'OWS','DWS','WS','WS/48','OBPM','DBPM','BPM','VORP','W','L','W/L%','PS/G','PA/G','SRS','Seed','MOV','ORtg','DRtg',
             'NRtg','MOV/A','ORtg/A','DRtg/A','NRtg/A','ORtg/100','DRtg/100']
for f in features :
    data[f'{f}_diff'] = data[f] - data[f'{f}_prev']

# Additional features
data['PTS_rel'] = [(pts-prev)/prev if prev > 0 else 0 for pts, prev in zip(data['PTS'], data['PTS_prev'])] # Points relative to previous year
data['PER_rel'] = (data['PER'] - data['PER_prev']) / data['PER_prev'] # PER relative to previous year
data['PTS_high_diff'] = data['PTS'] - data['PTS_high'] # Points difference with career high

### CUTS ###
# Previous seasons requirements (minutes and game)
data = data[data['MP_prev'] >= 5] # Minimum 5 minutes played in previous season
data = data[data['G_prev'] >= 10] # Minimum 10 games played in previous season
# Eligibility
data = data[data['PTS'] > data['PTS_high']] # Career high in points
data = data[(data['AST'] >= data['AST_high']) | (data['TRB'] >= data['TRB_high']) | (data['MP'] >= data['MP_high'])] # At least a career high in minutes, assists or rebounds

data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,NRtg_diff,MOV/A_diff,ORtg/A_diff,DRtg/A_diff,NRtg/A_diff,ORtg/100_diff,DRtg/100_diff,PTS_rel,PER_rel,PTS_high_diff
5,Jevon Carter,PG,27,MIL,39,30,25.2,2.8,6.9,0.403,...,-2.04,-2.11,-3.41,-1.43,-1.97,-36.0,-4.0,0.761905,0.020833,2.5
22,Josh Green,SG,22,DAL,25,1,20.3,2.4,4.1,0.583,...,-2.13,-1.51,2.75,4.49,-1.74,20.0,6.0,0.458333,0.126050,2.2
24,Luka Dončić,PG,23,DAL,37,37,36.9,11.4,22.8,0.499,...,-2.13,-1.51,2.75,4.49,-1.74,12.0,3.0,0.197183,0.250996,5.2
37,Jalen Johnson,SF,21,ATL,38,6,15.1,2.0,4.4,0.464,...,-2.71,-2.99,-4.07,-1.09,-2.99,13.0,-3.0,1.166667,0.115044,2.8
41,Onyeka Okongwu,C,22,ATL,39,12,23.9,3.7,6.1,0.619,...,-2.71,-2.99,-4.07,-1.09,-2.99,-10.0,0.0,0.109756,-0.141414,0.9


# Features, model

In [7]:
features = ['PTS_high','FG','VORP','FGA','OWS','WS','MP','PER','TRB','AST','OBPM','USG%']
features = [f + '_diff' for f in features] + ['PER_rel','lifetime']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [8]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/mip/mip_data.csv')

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15

In [9]:
# Set training data
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] >= season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [10]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1)) # Add rank
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'Tm'] + features + ['Share']]
res.head(5)

,Player,Tm,PTS_high_diff,FG_diff,VORP_diff,FGA_diff,OWS_diff,WS_diff,MP_diff,PER_diff,TRB_diff,AST_diff,OBPM_diff,USG%_diff,PER_rel,lifetime,Share
0,Tyrese Haliburton,IND,5.0,1.7,3.628205,3.4,4.051282,4.774359,-1.4,5.5,0.0,2.1,4.2,4.9,0.302198,3,0.295039
1,Lauri Markkanen,UTA,5.8,3.4,3.205000,4.5,6.320000,6.275000,3.2,8.2,2.6,0.6,4.8,5.4,0.557823,6,0.272359
2,Shai Gilgeous-Alexander,OKC,6.4,1.8,3.677778,1.8,6.538889,8.155556,1.0,5.8,-0.2,-0.3,2.6,2.2,0.277512,5,0.192629
3,Luka Dončić,DAL,5.2,1.5,3.851351,1.2,7.945946,8.356757,1.5,6.3,-0.3,0.0,2.3,1.2,0.250996,5,0.142660
4,Devin Vassell,SAS,7.1,2.5,1.279310,5.2,1.527586,0.658621,4.0,3.7,-0.3,1.7,2.7,6.2,0.278195,3,0.121574


In [11]:
res.to_csv(path + f'Results/mip/results_week_{week}.csv', index = None)